In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import os
from cv2 import imread
import cv2
import sys

from trainer import Trainer
from img_dataset import ImgDataset
from classifier import ResNet, BasicBlock, ConvFCNet, ConvFCNetv2, ConvFCNetv3

sys.path.append("../scheme/")

from predict_scheme import Scheme
import pillow_heif

### Load Trained Model

In [3]:
# model_filename = "model_resnet18_epoch10_acc8185"
# model_filename = "model_convfcnet_epoch10_acc8842" # acc=79.18%
# model_filename = "model_convfcnetv2_epoch10_acc9031" # acc=81.22%
model_filename = "model_convfcnetv3_epoch10_acc9271" # acc=81.22%

model = torch.load("./results/" + model_filename)

### Test on testing dataset

In [4]:
# Read training dataset
val_x = np.load("./data_chunk/test_x_chunk0.npy")
val_y = np.load("./data_chunk/test_y_chunk0.npy")[:, None]

print(type(val_x), val_x.shape)
print(type(val_y), val_y.shape)

<class 'numpy.ndarray'> (16703, 224, 224, 3)
<class 'numpy.ndarray'> (16703, 1)


In [5]:
test_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor()
])

val_dataset = ImgDataset(val_x, val_y, test_transforms)

In [6]:
trainer = Trainer(model = model, optimizer=None, num_epoch=0, loss=nn.BCELoss(), train_dataset=None, val_dataset=val_dataset, batch_size=64, device='cuda:0')

trainer.validation()

Val Acc: 0.927079 loss: 0.002858


### Using Scheme to predict large image

In [7]:
def accuracy(predict, target):
    return np.sum(np.equal(predict, target)) / target.size

In [8]:
scheme = Scheme(model, device='cuda:0')

images = os.listdir("../images/unprocessed_images")

dict = np.load("../images/unprocessed_images/y_test.npy", allow_pickle=True).item()

predicts = []
targets = []

for image in images:
    if ".HEIC" in image:
        heif_file = pillow_heif.open_heif("../images/unprocessed_images/{}".format(image))
        image_arr = np.asarray(heif_file)
    elif ".JPG" in image or ".jpg" in image:
        image_arr = imread("../images/unprocessed_images/{}".format(image))
    else:
        print(image)
        continue
        
    pred = scheme.make_prediction(image_arr)
    answer = dict.get(image, None)
    if answer != None:
        predicts.append(0 if pred == "apple" else 1)
        targets.append(answer)
    
        if (pred == "apple" and answer == 0) or (pred == "samsung" and answer == 1):
            print("{}: Correct!".format(image))
        else:
            print("{}: Incorrect!".format(image))

Number of small images: 18
06CBDB3B-BE6F-4BAC-998C-3030A247B462.jpg: Correct!
Number of small images: 221
20220206_175831.jpg: Correct!
Number of small images: 221
20220206_181421.jpg: Correct!
Number of small images: 221
20220709_201507.jpg: Correct!
20220930_163229.mp4
20220930_164147.mp4
Number of small images: 221
20221001_083124.jpg: Correct!
Number of small images: 221
20221001_083131.jpg: Correct!
Number of small images: 221
20221001_091644.jpg: Correct!
Number of small images: 221
20221001_105653.jpg: Correct!
Number of small images: 221
20221023_121044.jpg: Correct!
Number of small images: 221
20221023_121108.jpg: Correct!
Number of small images: 221
20221027_102910.jpg: Correct!
Number of small images: 221
20221027_122727.jpg: Correct!
Number of small images: 221
20221027_133040.jpg: Correct!
Number of small images: 221
20221027_153421.jpg: Incorrect!
Number of small images: 221
20221027_154517.jpg: Correct!
Number of small images: 221
20221027_154842.jpg: Correct!
Number of 

In [9]:
predicts = np.array(predicts)
targets = np.array(targets)

print("Number of testcases: {}".format(targets.size))

acc = accuracy(predicts, targets)
print("Accuracy: {}".format(acc))

Number of testcases: 197
Accuracy: 0.8121827411167513
